In [1]:
import pandas as pd

df = pd.read_excel('data/zkusenosti.xlsx')
cols = ['zkID', 'prodID', 'tagy', 'nadpis', 'zkusenost', 'lng']
lngs = ['pl', 'cz', 'hu', 'de', 'ro', 'es', 'ru', 'sk', 'en', 'it', 'us']

In [11]:
df2 = pd.read_excel('data/Produkty.xlsx')

filtered = [
    "bracelets_new",
    "eurasiapro",
    "extra",
    "astrafit",
    "hair color",
    "hodinky",
    "karta",
    "masážní",
    "nádobí",
    "reklama",
    "propagace",
    "samolepky",
    "sport-r",
    "termoterapie",
    "tiskoviny"
]
product_blacklist = [
    "DUO WHITE",
    "WHITE SET",
    "DUO WHITE",
    "WINDOW-CLEANER SET",
    "SNOW-BRUSH",
    "ICE-SCRAPER",
    "WHITE MAX",
    "WOOD MAX",
    "DUST MAX",
    "WINDOW-CLEANER"
    "WIRE TRIPLE STAR",
    "WIRE DOUBLE STAR",
    "SOAP mini",
    "SOAP WHITE",
    "SOAP BED",
]

import re
def extract_base_name(nazev):
    match = re.match(r'^(.*?)(\d+)$', nazev.strip())  # Extracts name and trailing number
    if match:
        return match.group(1).strip(), int(match.group(2))  # (Base Name, Number)
    return nazev.strip(), None  # Return name as-is if no number

df2_filtered = df2[
    (~df2['druh'].isin(filtered).fillna(False)) &  
    df2['show_it_CZ'].eq(1) & 
    ~df2['nazev'].str.endswith('SAMPLE', na=False) &  
    ~df2['nazev'].str.contains(r'\b\d+\s*[x×]\s*\d+(?:\s*cm)?\b', na=False) &
    ~df2['nazev'].str.match(r'^\w+$', na=False) & 
    ~df2['nazev'].str.contains("cm", case=True, na=False) &
    (~df2['nazev'].str.contains(",", case=False, na=False) | df2['nazev'].str.contains("CBD", case=False, na=False)) &
    ~df2['nazev'].str.contains("FLOOR", case=False, na=False) &
    ~df2['nazev'].str.contains("GLOVE", case=False, na=False) &
    ~df2['nazev'].isin(product_blacklist)
][['ID', 'nazev']]
df2_filtered = df2_filtered[~df2_filtered.duplicated(subset='nazev', keep='first')]
df2_filtered[['base_nazev', 'number']] = df2_filtered['nazev'].apply(lambda x: pd.Series(extract_base_name(x)))

df2_filtered = df2_filtered.sort_values(by='number', ascending=False).drop_duplicates(subset='base_nazev', keep='first').drop(columns=['base_nazev', 'number'])
df_cmp = df2_filtered[pd.to_numeric(df2_filtered['ID'], errors='coerce').notna()]

df_cmp.to_excel('just.xlsx', index=False)


In [48]:
import pandas as pd
import re
from rapidfuzz import process, fuzz

# Precompile regular expressions for speed.
punctuation_re = re.compile(r"[,.]")
digit_re = re.compile(r'\d')

def generate_phrases(text, max_words=4):
    """
    Generate a set of n-grams (from 1 to max_words) from the given text.
    Punctuation is removed from words, and only words with at least 3 characters are used.
    """
    words = text.split()
    phrases = set()
    for n in range(1, max_words + 1):
        for i in range(len(words) - n + 1):
            # Remove commas and dots from each word.
            cleaned_words = [punctuation_re.sub("", word) for word in words[i:i+n] if len(word) >= 3]
            if cleaned_words:
                phrase = " ".join(cleaned_words).lower()
                phrases.add(phrase)
    return phrases

def prepare_product_df(product_df):
    """
    Precompute several variants of the product name to avoid repeating
    regex/substring operations in the matching loop.
    """
    # Ensure lowercase version.
    product_df["nazev_lower"] = product_df["nazev"].str.lower()
    
    # Variant with trailing " star" removed if it exists.
    product_df["nazev_lower_nostar"] = product_df["nazev_lower"].apply(
        lambda s: s[:-5] if s.endswith(" star") else s
    )
    
    # Variant with digits removed.
    product_df["nazev_lower_nodigits"] = product_df["nazev_lower"].str.replace(r'\d+', '', regex=True)
    
    # Combination: no digits and no " star".
    product_df["nazev_lower_nodigits_nostar"] = product_df["nazev_lower_nostar"].str.replace(r'\d+', '', regex=True)
    
    return product_df

product_df = pd.read_excel("just.xlsx")
product_df = prepare_product_df(product_df)

def find_products_in_review(review, product_df, threshold=0.88):
    """
    For a given review, find product names that have a fuzzy matching score
    above the threshold. The threshold here is given as a fraction (e.g. 0.88),
    but RapidFuzz uses an integer scale [0, 100], so we convert accordingly.
    """
    # Normalize review text.
    review = review.lower().strip()
    phrases = generate_phrases(review)
    single_word_blacklist = {"life", "gold", "multi", ""}
    
    # Precompute candidate lists for all four matching variants.
    candidate_default      = product_df["nazev_lower"].tolist()
    candidate_nostar       = product_df["nazev_lower_nostar"].tolist()
    candidate_nodigits     = product_df["nazev_lower_nodigits"].tolist()
    candidate_nodigits_ns  = product_df["nazev_lower_nodigits_nostar"].tolist()
    
    # RapidFuzz scores are in [0, 100]. Convert threshold.
    score_cutoff = threshold * 100

    results = []  # We'll store matching results here.
    
    for phrase in phrases:
        has_digits = bool(digit_re.search(phrase))
        contains_star = "star" in phrase
        
        # Choose which candidate list to use based on the phrase.
        if not contains_star and phrase not in single_word_blacklist:
            candidate_list = candidate_nodigits_ns if not has_digits else candidate_nostar
        else:
            candidate_list = candidate_nodigits if not has_digits else candidate_default
        
        # Use RapidFuzz to find the best matching product name.
        best = process.extractOne(
            phrase,
            candidate_list,
            scorer=fuzz.ratio,
            score_cutoff=score_cutoff  # Only return a match if above cutoff.
        )
        
        if best:
            match_str, score, idx = best
            # Retrieve the product row that corresponds to the matched candidate.
            row = product_df.iloc[idx]
            results.append({
                "ID": row["ID"],
                "nazev": row["nazev"],
                "phrase": phrase,
                # Convert score to a 0-1 scale (optional)
                "score": score / 100.0
            })
    
    # Build the results DataFrame (if no results, returns an empty DataFrame).
    result_df = pd.DataFrame(results, columns=["ID", "nazev", "phrase", "score"])
    
    # Sort by score and drop duplicate product names, keeping the best match.
    if not result_df.empty:
        result_df = result_df.sort_values(by="score", ascending=False) \
                             .drop_duplicates(subset="nazev", keep="first")
    return result_df

# Test example
review_text = ("Vzhledem k mé myotomické dystrofii II, JOINT MOBILITY STAR mi pomáhá nejen od bolesti nohou "
               "ale i od křečí. MULTI STAR také skvěle funguje pro můj imunitní systém.")

print(find_products_in_review(review_text, product_df, threshold=0.88))

     ID                nazev               phrase  score
1  1845           MULTI STAR           multi star    1.0
4  1070  JOINT MOBILITY STAR  joint mobility star    1.0


In [28]:
for review_text in df['zkusenost'].sample(1000):
    print(f"Review: {review_text}")
    print(find_products_in_review(review_text, product_df))
    print("\n---------------\n")
   

Review: Az OCUCAR,S bizonyítottan változatlan, nem romló állapotban tartja retinánkat a hosszú műtéti várakozás alatt. 6 óránként csepegtettük, Hyaluronic 2x20 ml és Carnosine star3x3 kombinálva
     ID            nazev      phrase  score
0  1777   CARNOSINE STAR   carnosine    1.0
2  7670  HYALURONIC STAR  hyaluronic    1.0

---------------

Review: Az unokámnak szúnyogcsipésre haszáltuk. A csípés csúnyán bedagadt, égett, fájt._x000D_
az Effective Star Basickel rövid időn belül megszüntek a tünetek.
     ID                 nazev                   phrase     score
0  5767  EFFECTIVE STAR BASIC  effective star basickel  0.930233

---------------

Review: Az Aloe Vera gél jó tulajdonságait már régota ismerem és szeretem. Ezt a terméket azért is kedvelem, mert bátran vihetem magammal bárhova. Kis helyen elfér, és nagyon jó hatása van az egész szervezetre.
     ID               nazev         phrase     score
0  1124  ALOE VERA GEL STAR  aloe vera gél  0.923077

---------------

Review: A Q

In [171]:
system = """
Your job is create tags and a title for a given text, as well as find the product name. The text is always some kind of review for a dietary supplement product.
The title must simply encapsulate what the text is about in at most 5 words. The titles should be a softly unique so there aren't too many duplicates. The product names are usually within the review text but not always, in which case there is no product name to find.
The language of the review is in {lng} and you must also return the tags and title in that language!

The tags are generated by asking the following questions.
- What was the problem?
- What improvements did the user experience?
- Which part of the body is the review about?
Since these tags are used as extract string matches in a search query, you must generate multiple tags of the same thing if it makes sense. Here is an example:
**Review sample:** 'Vynikající kombinovaný antioxidant. Výborné zkušenosti mám u osob s rodovým rizikem infarktu, a to i u osob, které infarkt již prodělali. Vhodný pro harmonizaci funkce jater, v některých případech zlepšuje výsledky jaterních testů a může pomoci i při vysoké hladině cholesterolu. Pomáhá i lidem s oslabením imunity a též při některých pohybových potížích. Vhodný jako součást preventivní kůry a podpůrných opatření u některých nádorových onemocněních.'

**Your sample output"**:
{{
    "nadpis": "Silný antioxidant pro srdce, játra a imunitu",
    "tagy": "myotonická dystrofie II, bolest nohou, křeče, pohybové potíže, úleva od bolesti, snížení křečí, zlepšení mobility, schopnost chůze, nohy, svaly, klouby",
}} 


**Review:**
{review}
"""

In [172]:
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

import os

LLM = AzureChatOpenAI(
    azure_endpoint = 'https://starlife-swe.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview',
    deployment_name = 'gpt-4o-mini',
    openai_api_key = '9596f5c3ab2241cd9103227f475d6c13',
    openai_api_version = "2024-08-01-preview",
    temperature=0,
    verbose=True,
)

class Output(BaseModel):
    tagy: str = Field(..., description="Comma separated string of tags relevant to the review")
    nadpis: str = Field(..., description="Title summarizing the review")


product_df = pd.read_excel("just.xlsx")
product_df = prepare_product_df(product_df)
def agent(row):
    review = row['zkusenost']
    lng = row['lng']

    prompt = ChatPromptTemplate.from_messages(
        [
            ('system', system),
        ]
    )
    structured_llm = LLM.with_structured_output(Output)
    chain = prompt | structured_llm
    response = dict(chain.invoke({'lng': lng, 'review': review}))

    response['zkusenost'] = review
    return response
    

In [173]:
zkusenost, lng = df[['zkusenost', 'lng']].sample(1).iloc[0]
res = agent({"zkusenost": zkusenost, 'lng': lng})
res['prodID'] = find_products_in_review(res['zkusenost'], product_df)['ID'].astype(str).str.cat(sep=",")
res

{'tagy': 'puffadás, méregtelenítés, panaszmentesség, emésztés, inulin, bélrendszer, emésztési problémák',
 'nadpis': 'Hatékony méregtelenítés és emésztés',
 'zkusenost': 'Tavaszi méregtelenítéshez próbáltam ki, elmúltak a puffadásos panaszok. Mellé még az Inulint is fogyasztottam, azóta panaszmentes vagyok',
 'prodID': '4580'}

In [170]:
res

{'tagy': 'orrfolyás megszüntetése,könnyezés megszüntetése,szemviszketés megszüntetése,gyors hatás,immunitás javítása',
 'nadpis': 'Gyors megoldás allergiás tünetekre',
 'zkusenost': 'Kb. 14 nap alatt jelentkeztek a gyógyulás jelei, megszűnt az orr folyás, a könnyezés és a szem viszketés. Ezt a terméket az Immunity Starral együtt fejti ki legjobban a gyors hatást.',
 'prodID': '1180'}

In [202]:
import pandas as pd
import re

def check_df_integrity(original_df: pd.DataFrame, modified_df: pd.DataFrame) -> dict:
    errors = {}

    # 1. Check if the number of rows and columns are the same
    if original_df.shape != modified_df.shape:
        errors["shape_mismatch"] = {
            "original_shape": original_df.shape,
            "modified_shape": modified_df.shape
        }

    # 2. Check if all columns are present and in the same order
    if list(original_df.columns) != list(modified_df.columns):
        errors["column_mismatch"] = {
            "missing_in_modified": list(set(original_df.columns) - set(modified_df.columns)),
            "extra_in_modified": list(set(modified_df.columns) - set(original_df.columns))
        }

    # Helper functions for format validation of modifiable columns:
    def is_valid_tagy(value) -> bool:
        """
        Validates that 'tagy' is a string that is either empty or a comma+space separated list.
        Valid examples: "", "tagb", "tagb, tagc, taga"
        Missing values (NaN) are treated as valid.
        """
        if pd.isna(value):
            return True
        if not isinstance(value, str):
            return False
        if value == "":
            return True
        return True

    def is_valid_nadpis(value) -> bool:
        """
        Validates that 'nadpis' is a string.
        Missing values (NaN) are treated as valid.
        """
        if pd.isna(value):
            return True
        return isinstance(value, str)

    def is_valid_prodid(value) -> bool:
        """
        Validates that 'prodID' is a string that is either empty or a comma+space separated list of integers.
        Valid examples: "", "100", "100, 101, 102"
        Missing values (NaN) are treated as valid.
        """
        if pd.isna(value):
            return True
        if not isinstance(value, str):
            return False
        if value == "":
            return True
        tokens = value.split(", ")
        # Ensure that the delimiter is correctly used (re-join should match original string)
        if ", ".join(tokens) != value:
            return False

        return True

    modifiable_columns = {"tagy", "nadpis", "prodID"}

    # 3. Check if any non-modifiable columns have changed
    for col in original_df.columns:
        if col not in modifiable_columns:
            if not original_df[col].equals(modified_df[col]):
                errors.setdefault("data_mismatch", []).append(col)

    # 4. Validate the format of the modifiable columns in modified_df
    if "tagy" in modified_df.columns:
        for index, value in modified_df["tagy"].items():
            if not is_valid_tagy(value):
                errors.setdefault("tagy_format_error", {})[index] = value

    if "nadpis" in modified_df.columns:
        for index, value in modified_df["nadpis"].items():
            if not is_valid_nadpis(value):
                errors.setdefault("nadpis_format_error", {})[index] = value

    if "prodID" in modified_df.columns:
        for index, value in modified_df["prodID"].items():
            if not is_valid_prodid(value):
                errors.setdefault("prodID_format_error", {})[index] = value

    return errors if errors else {"status": "Integrity check passed"}


In [174]:
import numpy as np
df_new = df.copy()
ilocs = np.random.randint(0, df.shape[0]-3, 5)

for index, row in df_new.loc[ilocs].iterrows():
    zkusenost, lng = row[['zkusenost', 'lng']]
    res = agent({"zkusenost": zkusenost, 'lng': lng})
    res['prodID'] = find_products_in_review(res['zkusenost'], product_df)['ID'].astype(str).str.cat(sep=", ")
    df_new.loc[index, ['prodID', 'nadpis', 'tagy']] = [res['prodID'], res['nadpis'], res['tagy']]


In [203]:
check_df_integrity(df, df)

{'status': 'Integrity check passed'}